In [ ]:
#------------------------Gaurav Verma(202051071)-------------------------------#
import os

class KBC(object):
  def __init__(self, no_of_questions, probabilities, rewards):
    self.no_of_questions = no_of_questions
    self.probabilities = probabilities
    self.rewards = rewards
  def startState(self):
    return 1
  def endState(self, state):
    return state == self.no_of_questions + 1
  def actions(self, state):
    if self.endState(state):
        return []
    else:
        return ('quit', 'move')
  def succesor_Probability_Reward(self, state, action):
    if action == 'move':
        probability = self.probabilities[state - 1]
        reward = self.rewards[state - 1]
        return [(state + 1, probability, reward), (self.no_of_questions + 1, 1 - probability, 0)]
    else:
        return [(self.no_of_questions + 1, 1, self.rewards[state - 1])]
  def discount(self):
    return 0.9
  def state(self):
    return range(1,self.no_of_questions+2)
    


def valueIteration(mdp, e=0.001):
    V = {s: 0.0 for s in mdp.state()}
    while True:
        delta = 0
        for s in mdp.state():
            if mdp.endState(s):
                continue
            MaxValue = -float('inf')
            for a in mdp.actions(s):
                val = 0
                for NextState, probability, reward in mdp.succesor_Probability_Reward(s, a):
                    val += probability * (reward + mdp.discount() * V[NextState])
                MaxValue = max(MaxValue, val)
            delta = max(delta, abs(MaxValue - V[s]))
            V[s] = MaxValue
        if delta < e:
            break
    pi = {}
    for s in mdp.state():
        if mdp.endState(s):
            pi[s] = None
        else:
            MaxValue = -float('inf')
            best_action = None
            for a in mdp.actions(s):
                val = 0
                for NextState, probability, reward in mdp.succesor_Probability_Reward(s, a):
                    val += probability * (reward + mdp.discount() * V[NextState])
                if val > MaxValue:
                    MaxValue = val
                    best_action = a
            pi[s] = best_action
    return V, pi

Questions = 10
given_prob = [0.99, 0.9, 0.8, 0.7, 0.6, 0.5, 0.4, 0.3, 0.2, 0.1]
given_rewards = [100, 500, 1000, 5000, 10000, 50000, 100000, 500000, 1000000, 5000000]
kbc_MDP = KBC(Questions, given_prob, given_rewards)

V, P = valueIteration(kbc_MDP)
print("Value Iteration", V)
print("Policy",P)

Value Iteration {1: 26780.771381760005, 2: 29945.87136000001, 3: 36414.656, 4: 49464.799999999996, 5: 72960.0, 6: 124000.0, 7: 220000.0, 8: 500000.0, 9: 1100000.0, 10: 5000000.0, 11: 0.0}
Policy {1: 'move', 2: 'move', 3: 'move', 4: 'move', 5: 'move', 6: 'move', 7: 'move', 8: 'quit', 9: 'move', 10: 'quit', 11: None}


In [ ]:
def policyIteration(mdp, e=0.001):
    V = {s: 0.0 for s in mdp.state()}
    pi = {s: mdp.actions(s)[0] for s in mdp.state() if not mdp.endState(s)}
    while True:
        while True:
            delta = 0
            for s in mdp.state():
                if mdp.endState(s):
                    continue
                val = 0
                for NextState, probability, reward in mdp.succesor_Probability_Reward(s, pi[s]):
                    val += probability * (reward + mdp.discount() * V[NextState])
                delta = max(delta, abs(val - V[s]))
                V[s] = val
            if delta < e:
                break
        policy_stable = True
        for s in mdp.state():
            if mdp.endState(s):
                continue
            old_action = pi[s]
            MaxValue = -float('inf')
            best_action = None
            for a in mdp.actions(s):
                val = 0
                for NextState, probability, reward in mdp.succesor_Probability_Reward(s, a):
                    val += probability * (reward + mdp.discount() * V[NextState])
                if val > MaxValue:
                    MaxValue = val
                    best_action = a
            pi[s] = best_action
            if old_action != best_action:
                policy_stable = False
        if policy_stable:
            break
    return V, pi

Questions = 10
given_prob = [0.99, 0.9, 0.8, 0.7, 0.6, 0.5, 0.4, 0.3, 0.2, 0.1]
given_rewards = [100, 500, 1000, 5000, 10000, 50000, 100000, 500000, 1000000, 5000000]
Kbc_MDP = KBC(Questions,given_prob, given_rewards)

V ,P = policyIteration(Kbc_MDP)

print("Policy",P)
print("Value Iteration",V)

Policy {1: 'move', 2: 'move', 3: 'move', 4: 'move', 5: 'move', 6: 'move', 7: 'move', 8: 'quit', 9: 'move', 10: 'quit'}
Value Iteration {1: 26780.771381760005, 2: 29945.87136000001, 3: 36414.656, 4: 49464.799999999996, 5: 72960.0, 6: 124000.0, 7: 220000.0, 8: 500000.0, 9: 1100000.0, 10: 5000000.0, 11: 0.0}


In [ ]:
import random
def generateSARS(Kbc_MDP, pi, num_sequences=1, max_steps=100):
    sequences = []
    for i in range(num_sequences):
        states = Kbc_MDP.startState()
        sequence = []
        for t in range(max_steps):
            if Kbc_MDP.endState(states):
                break
            action = pi[states]
            next_states, probabilities, rewards = zip(*Kbc_MDP.succesor_Probability_Reward(states, action))
            next_state = random.choices(next_states, probabilities)[0]
            reward = rewards[next_states.index(next_state)]
            sequence.append((states, action, reward, next_state))
            state = next_state
        sequences.append(sequence)
    return sequences
sequences = generateSARS(Kbc_MDP, P, num_sequences=10, max_steps=10)
for sequence in sequences:
    print(sequence)
    print("\n")

[(1, 'move', 100, 2), (1, 'move', 100, 2), (1, 'move', 100, 2), (1, 'move', 100, 2), (1, 'move', 100, 2), (1, 'move', 100, 2), (1, 'move', 100, 2), (1, 'move', 100, 2), (1, 'move', 100, 2), (1, 'move', 100, 2)]


[(1, 'move', 100, 2), (1, 'move', 100, 2), (1, 'move', 100, 2), (1, 'move', 100, 2), (1, 'move', 100, 2), (1, 'move', 100, 2), (1, 'move', 100, 2), (1, 'move', 100, 2), (1, 'move', 100, 2), (1, 'move', 100, 2)]


[(1, 'move', 100, 2), (1, 'move', 100, 2), (1, 'move', 100, 2), (1, 'move', 100, 2), (1, 'move', 100, 2), (1, 'move', 100, 2), (1, 'move', 100, 2), (1, 'move', 100, 2), (1, 'move', 100, 2), (1, 'move', 100, 2)]


[(1, 'move', 100, 2), (1, 'move', 100, 2), (1, 'move', 100, 2), (1, 'move', 100, 2), (1, 'move', 100, 2), (1, 'move', 100, 2), (1, 'move', 100, 2), (1, 'move', 100, 2), (1, 'move', 100, 2), (1, 'move', 100, 2)]


[(1, 'move', 100, 2), (1, 'move', 100, 2), (1, 'move', 100, 2), (1, 'move', 100, 2), (1, 'move', 100, 2), (1, 'move', 100, 2), (1, 'move', 100, 2), 

In [ ]:
# Monto Carlo 
import random

class MontoCarlo(object):

    def __init__(self, Questions, Prob_Rewards):
        self.Questions = Questions
        self.Prob_Rewards = Prob_Rewards

    def play(self, policy):
        state = self.InitState()
        done = False
        rewards = []
        while not done:
            action = policy(state)
            next_state, reward, done = self.SuccProbReward(state, action)[0]
            rewards.append(reward)
            state = next_state
        cumulative_rewards = [sum(rewards[i:]) for i in range(len(rewards))]
        episodes = [(self.InitState(), None, 0)]
        for i in range(len(rewards)):
            episodes.append((i+1, policy(i+1), cumulative_rewards[i]))
        return episodes

    def InitState(self):
        return 1

    def EndState(self, state):
        return state == self.Questions + 1

    def Actions(self, state):
        if self.EndState(state):
            return []
        else:
            return ['Stay', 'Quit']

    def SuccProbReward(self, state, action):
        if action == 'Stay':
            if state > self.Questions:
                return [(state, 0, True)]
            else:
                prob, reward = self.Prob_Rewards[state]
                if random.random() <= prob:
                    return [(state+1, reward, False)]
                else:
                    return [(self.Questions+1, 0, True)]
        elif action == 'Quit':
            if state > self.Questions:
                return [(state, 0, True)]
            else:
                reward = self.Prob_Rewards[state][1]
                return [(self.Questions+1, reward, True)]

    def discount(self):
        return 0.9

    def states(self):
        return range(1, self.Questions+2)


def random_policy(state):
    actions = ['Stay', 'Quit']
    return random.choice(actions)

env = MontoCarlo(10, [(0.99, 100),(0.9, 500),(0.8, 1000),(0.7, 5000), (0.6, 10000), (0.5, 50000), (0.4, 100000), (0.3, 500000), (0.2, 1000000), (0.1, 5000000)])

episode = env.play(random_policy)

print(episode)


[(1, None, 0), (1, 'Stay', 500)]


In [ ]:
import numpy as np

Questions = 10
Prob_Rewards = [(0.99, 100),(0.9, 500),(0.8, 1000),(0.7, 5000), (0.6, 10000), (0.5, 50000), (0.4, 100000), (0.3, 500000), (0.2, 1000000), (0.1, 5000000)]

num_states = Questions + 1
num_actions = 2

R = np.zeros(num_states).astype(int)
for s in range(num_states):
    if s == num_states - 1:
        R[s] = 0
    else:
        R[s] = Prob_Rewards[s][1]


P1 = np.array([[0.99, 0.01, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                [0, 0.9, 0.1, 0, 0, 0, 0, 0, 0, 0, 0],
                [0, 0, 0.8, 0.2, 0, 0, 0, 0, 0, 0, 0],
                [0, 0, 0, 0.7, 0.3, 0, 0, 0, 0, 0, 0],
                [0, 0, 0, 0, 0.6, 0.4, 0, 0, 0, 0, 0],
                [0, 0, 0, 0, 0, 0.5, 0.5, 0, 0, 0, 0],
                [0, 0, 0, 0, 0, 0, 0.4, 0.6, 0, 0, 0],
                [0, 0, 0, 0, 0, 0, 0, 0.3, 0.7, 0, 0],
                [0, 0, 0, 0, 0, 0, 0, 0, 0.2, 0.8, 0],
                [0, 0, 0, 0, 0, 0, 0, 0, 0, 0.1, 0.9],
                 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]])

print("Transition matrix : \n")
for i in range(num_states):
    for j in range(num_states):
        print(P1[i][j], end = "   ")
    print("\n")
print("Rewards Vector : \n")
print(R)



gamma = 0.9

A = np.eye(num_states) - gamma * P1
b = R
V = np.linalg.inv(A).dot(b)

print("\n Value function : \n")
print(V)



Transition matrix : 

0.99   0.01   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   

0.0   0.9   0.1   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   

0.0   0.0   0.8   0.2   0.0   0.0   0.0   0.0   0.0   0.0   0.0   

0.0   0.0   0.0   0.7   0.3   0.0   0.0   0.0   0.0   0.0   0.0   

0.0   0.0   0.0   0.0   0.6   0.4   0.0   0.0   0.0   0.0   0.0   

0.0   0.0   0.0   0.0   0.0   0.5   0.5   0.0   0.0   0.0   0.0   

0.0   0.0   0.0   0.0   0.0   0.0   0.4   0.6   0.0   0.0   0.0   

0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.3   0.7   0.0   0.0   

0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.2   0.8   0.0   

0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.1   0.9   

0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   1.0   

Rewards Vector : 

[    100     500    1000    5000   10000   50000  100000  500000 1000000
 5000000       0]

 Value function : 

[  63648.10943705  759738.21429313 1598336.23017438 2480745.24693792
 3381021.26432234 4292416.05